In [54]:
import numpy as np 
import scipy

In [30]:
#41連立方程式の表現
#43連立方程式を解く
A1=np.array([
    [1,5],
    [5,6]
])
B1=np.array([
    [3],
    [4]
])
c=[[x],
   [y]]
print(np.linalg.solve(A1,B1))

[[0.10526316]
 [0.57894737]]


In [25]:
#42連率方程式の表現２
#43連立方程式を解く
A2=np.array([
    [1,5,3],
    [5,4,2],
    [4,3,5]
])
B2=np.array([
    [3],
    [4],
    [7]
])
c=[[x],
   [y],
   [z]]
print(np.linalg.solve(A2,B2))

[[ 0.48648649]
 [-0.16216216]
 [ 1.10810811]]


In [26]:
#行列のランク２
def row_swap(A,i,j):
    A[i],A[j]=A[j],A[i]

def row_scale(A,i,k):
    if k==0:
        raise ValueError('k must be non-zero')
    A[i]=[k*x for x in A[i]]

def row_add(A,src,dest,k):
    A[dest]=[d+k*s for d,s in zip(A[dest],A[src])]

def gaussian_elmination(mat):
    m,n=len(mat),len(mat[0])
    pivot_cols=[]
    row=0
    for col in range(n):
        sel=max(range(row,m),key=lambda r: abs(mat[r][col]))
        if abs(mat[sel][col])<1**(-12):
            continue
        row_swap(mat,row,sel)
        pivot_cols.append(col)
        
        pivot=mat[row][col]
        mat[row]=[x/pivot for x in mat[row]]
        
        for r in range(m):
            if r==row:
                continue
            factor=mat[r][col]
            row_add(mat,row,r,-factor)
        row+=1
        if row==m:
            break
    return pivot_cols

In [33]:
A3 = np.concatenate((A1, B1), axis=1)
print(np.linalg.matrix_rank(A3))
print(gaussian_elmination(A3))
A4=np.concatenate((A2,B2),axis=1)
print(gaussian_elmination(A4))

2
[0, 1]
[0, 1, 2]


In [52]:
#46,47クラメルの公式
def cramer(A,B):
    detA=np.linalg.det(A)
    tA=[r for r in zip(*A)]
    tB=[r for r in zip(*B)]
    res=[]
    for i in range(len(A)):
        p=tA[i]
        tA[i]=tB[0]
        ttA=[r for r in zip(*tA)]
        res.append(np.linalg.det(ttA)/detA)
        tA[i]=p
    return res

A=[[1,3],[4,2]]
B=[[2],[6]]
C=cramer(A,B)
print(C)
A2=[[1,2,3],[3,2,1],[2,3,1]]
B2=[[5],[6],[7]]
C2=cramer(A2,B2)
print(C2)

[np.float64(1.4000000000000001), np.float64(0.19999999999999996)]
[np.float64(0.7499999999999999), np.float64(1.75), np.float64(0.24999999999999986)]


In [56]:
#48,49 LU分解
Alu=scipy.linalg.lu(A2)
for mx in Alu:
    print(mx)

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[1.         0.         0.        ]
 [0.66666667 1.         0.        ]
 [0.33333333 0.8        1.        ]]
[[3.         2.         1.        ]
 [0.         1.66666667 0.33333333]
 [0.         0.         2.4       ]]


In [58]:
from scipy.linalg import lu_factor, lu_solve

# 係数行列 A
A = np.array([[2, 5, 8],
              [5, 2, 2],
              [7, 5, 6]])

# 定数項ベクトル b
b = np.array([23, 14, 32])

# 1. 行列 A を LU 分解する (lu_factor を使用)
# lu_factor は (LUを組み合わせた行列, ピボット情報) のタプルを返す
lu_and_piv = lu_factor(A)

# 2. LU 分解の結果と b を使って連立方程式を解く
x = lu_solve(lu_and_piv, b)
x2 = np.linalg.solve(A,b)
# 解を表示
print("連立方程式の解 x:\n", 'scipy:',x,'numpy:',x2)

# 検算: A @ x が b に等しくなるか確認
print("A @ x:\n", A @ x)
print("Is A @ x close to b?:", np.allclose(A @ x, b))

連立方程式の解 x:
 scipy: [-0.5  14.   -5.75] numpy: [-0.5  14.   -5.75]
A @ x:
 [23. 14. 32.]
Is A @ x close to b?: True


In [ ]:
#50連立方程式の数値誤差
#観察済み